In [1]:
!pip install -q langchain_experimental langchain_core google-generativeai==0.3.1 google-ai-generativelanguage==0.4.0 langchain-google-genai
!pip install -q "langchain[docarray]"

In [2]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken chromadb lark

In [3]:
!pip -q install sentence_transformers
!pip -q install -U FlagEmbedding

In [4]:
import os

os.environ['GOOGLE_API_KEY']=""

In [5]:
!pip show langchain-google-genai

Name: langchain-google-genai
Version: 0.0.6
Summary: An integration package connecting Google's genai package and LangChain
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: google-generativeai, langchain-core
Required-by: 


In [6]:
import os
import google.generativeai as genai

genai.configure(api_key="")

**Parent Document Retriever**

1. You return a Full document for small chunks if the size of original document is lesser to look up for
2. You also return a bigger chunk for smaller chunks to look up for


In [7]:
!pip install -q PyPDF2

In [8]:
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.document_loaders import TextLoader

from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI

from PyPDF2 import PdfReader

embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [13]:
!mkdir -p DataInfo
!unzip -q /content/langchain_blog_posts.zip -d DataInfo

In [15]:
loaders = [
    TextLoader('/content/DataInfo/blog.langchain.dev_announcing-langsmith_.txt'),
    TextLoader('/content/DataInfo/blog.langchain.dev_announcing-langsmith_?ref=commandbar.ghost.io.txt'),
    TextLoader('/content/DataInfo/blog.langchain.dev_benchmarking-question-answering-over-csv-data_.txt')
]
docs = []
for l in loaders:
    docs.extend(l.load())

In [16]:
len(docs)

3

In [17]:
docs[0]

Document(page_content='URL: https://blog.langchain.dev/announcing-langsmith/\nTitle: Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications\n\nLangChain exists to make it as easy as possible to develop LLM-powered applications.\n\nWe started with an open-source Python package when the main blocker for building LLM-powered applications was getting a simple prototype working. We remember seeing Nat Friedman tweet in late 2022 that there was “not enough tinkering happening.” The LangChain open-source packages are aimed at addressing this and we see lots of tinkering happening now (Nat agrees)–people are building everything from chatbots over internal company documents to an AI dungeon master for a Dungeons and Dragons game.\n\nThe blocker has now changed. While it’s easy to build a prototype of an application in ~5 lines of LangChain code, it’s still deceptively hard to take an application from prototype to production. The main is

# **Retrieving Full Docments rathere than Chunks**

Here, We will the return Original documents aren't too big in length

In [18]:
child_spitter = RecursiveCharacterTextSplitter(chunk_size=400)

vectorstore = Chroma(
    collection_name="Full_Documents",
    embedding_function=embeddings
)

store = InMemoryStore()

full_doc_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_spitter,
)

In [19]:
full_doc_retriever.add_documents(docs, ids=None)

In [20]:
list(store.yield_keys())

['c1af498f-da83-47f1-92df-3ac0d9b4f53c',
 'fefc224c-c715-4f4d-bda0-1599ccc7ade3',
 'e8ae0892-ed69-4216-a906-6fa3fbfb34e1']

In [21]:
sub_docs = vectorstore.similarity_search("what is langsmith", k=2)

In [22]:
len(sub_docs)

2

In [23]:
print(sub_docs[0].page_content)

A unified platform


In [24]:
retrieved_docs = full_doc_retriever.get_relevant_documents("what is langsmith")

In [25]:
len(retrieved_docs[0].page_content)

11652

In [26]:
retrieved_docs[0].page_content

'URL: https://blog.langchain.dev/announcing-langsmith/\nTitle: Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications\n\nLangChain exists to make it as easy as possible to develop LLM-powered applications.\n\nWe started with an open-source Python package when the main blocker for building LLM-powered applications was getting a simple prototype working. We remember seeing Nat Friedman tweet in late 2022 that there was “not enough tinkering happening.” The LangChain open-source packages are aimed at addressing this and we see lots of tinkering happening now (Nat agrees)–people are building everything from chatbots over internal company documents to an AI dungeon master for a Dungeons and Dragons game.\n\nThe blocker has now changed. While it’s easy to build a prototype of an application in ~5 lines of LangChain code, it’s still deceptively hard to take an application from prototype to production. The main issue that we see today 

# **Retrieving Larger Chunks**

In [27]:
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
)

child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
)

vectorstore = Chroma(
    collection_name="split_parents",
    embedding_function=embeddings
)

store = InMemoryStore()

In [28]:
big_chunks_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [29]:
big_chunks_retriever.add_documents(docs)

In [30]:
len(list(store.yield_keys()))

25

In [31]:
sub_docs = vectorstore.similarity_search("what is langsmith")

In [32]:
len(sub_docs)

4

In [33]:
print(sub_docs[0].page_content)

A unified platform


In [34]:
retrieved_docs = big_chunks_retriever.get_relevant_documents("what is langsmith")

In [35]:
len(retrieved_docs)

4

In [36]:
len(retrieved_docs[0].page_content)

1955

In [37]:
print(retrieved_docs[0].page_content)

“Thanks to Langchain smith we were able to analyze our LLM calls, understand the performance of the different chain methods ( stuff vs reduce) for QA and improve upon it. It even helped us debug and understand errors we made. We are consistently using it to improve our prompt engineering and look forward to the new features,” said Stan Girard, Head of GenAI at Theodo and creator of Quivr.



A unified platform

While each of these product areas provide unique value, often at a specific point in time in the development process, we believe a great deal of the long term impact of LangSmith will come from having a single, fully-integrated hub to do this work from. We see teams with all kinds of Rube Goldberg-machine-like processes for managing their LLM applications, and we want to make that a thing of the past.

As a very simple example, we considered it to be table stakes for LangSmith to help users easily create datasets from existing logs and use them immediately for testing and evalua

In [38]:
print(retrieved_docs[2].page_content)

In another example of debugging in action, we partnered with DeepLearningAI to equip learners in the recently-released LangChain courses with access to LangSmith. This allowed students to easily visualize the exact sequence of calls, and the inputs and outputs at each step in the chain with precision. Students can understand exactly what the chains, prompts, and LLMs were doing, which helps build intuition as they learn to create new and more sophisticated applications.



Testing

One of the main questions we see developers grapple with is: “If I change this chain/prompt, how does that affect my outputs?” The most effective way to answer this question is to curate a dataset of examples that you care about, and then run any changed prompts/chains over this dataset. LangSmith first makes it easy to create these datasets from traces or by uploading datasets you’ve curated manually. You can then easily run chains and prompts over those data sets.

The first helpful step is simply manually

In [39]:
from langchain.chains import RetrievalQA
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.llms import GooglePalm

qa = RetrievalQA.from_chain_type(
    llm = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.8,convert_system_message_to_human=True),
    chain_type="stuff",
    retriever=big_chunks_retriever
)

query = "What is Langsmith?"

qa.run(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Langsmith is a unified platform for debugging, testing, evaluating, and monitoring your LLM applications. It is designed to help developers close the gap between prototype and production. Langsmith provides a clear interface for letting developers easily see the inputs and outputs for each data point, as without that visibility they cannot build up that intuition.'